# Vanilla models test

- Artificial Neural Networks
- Decision Tree Classifier
- Random Forest
- Gradient Boosting

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9707045019894422791
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1745302324
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5442142398385891752
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## Dataset loading

In [4]:
train_dir = 'data/real_vs_fake/real-vs-fake/train'
valid_dir = 'data/real_vs_fake/real-vs-fake/valid'
test_dir =  'data/real_vs_fake/real-vs-fake/test'

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    data_format='channels_last',
    rotation_range = 45,
    zoom_range = (.95,1.05),
    horizontal_flip=True,
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
batch_size = 32
image_height = 128
image_width = 128

# Create the generators for training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    shuffle=True,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',     # Set the class mode to 'binary' since have two classes
)

valid_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


fake images have label 0 and real images have label 1

In [7]:
classes = list(train_generator.class_indices.keys())
classes

['fake', 'real']

In [28]:
# Checking some images
images, labels = train_generator.next()

# Display the images along with their labels
fig, axes = plt.subplots(4, 4, figsize=(12, 12))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.axis('off')

    if int(labels[i]) == 0:
        title_color = 'red'
    else:
        title_color = 'green'

    ax.set_title(classes[int(labels[i])], color=title_color)

plt.show()

ModuleNotFoundError: No module named 'matplotlib_inline'

## Convolutional neural network

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [10]:
# Model
cnn = Sequential()

# Convolutional layers
cnn.add(Conv2D(16, (5, 5), activation='relu',padding='same',strides=(1,1),input_shape=(image_height, image_width, 3)))
cnn.add(Conv2D(32, (3, 3), activation='relu',padding='same',strides=(2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu',padding='same',strides=(1,1)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(2,2)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(1,1)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(2,2)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(1,1)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(2,2)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(1,1)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(2,2)))
cnn.add(Conv2D(128, (3, 3), activation='relu',padding='same',strides=(1,1)))
cnn.add(Conv2D(128, (4, 4), activation='relu',padding='valid',strides=(1,1)))

# # Flatten layer to feed into dense Dense
cnn.add(Flatten())

# # Dense 
# cnn.add(Dropout(0.5))
# cnn.add(Dense(128, activation='relu'))
# cnn.add(Dropout(0.5))
# cnn.add(Dense(64, activation='relu'))
# cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation='sigmoid'))  # Adjust the number of classes

In [11]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
cnn.compile(optimizer='adam'
            ,loss='binary_crossentropy'
            ,metrics=['acc',f1_m,precision_m, recall_m])
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      1216      
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 128)       1

In [13]:
cnn_history = cnn.fit(train_generator, epochs=10,validation_data=valid_generator)

Epoch 1/10
3125/3125 [==============================] - 1450s 462ms/step - loss: 0.6932 - acc: 0.4984 - f1_m: 0.3783 - precision_m: 0.2855 - recall_m: 0.5725 - val_loss: 0.6932 - val_acc: 0.5000 - val_f1_m: 0.6621 - val_precision_m: 0.5000 - val_recall_m: 1.0000
Epoch 2/10
3125/3125 [==============================] - 922s 295ms/step - loss: 0.6932 - acc: 0.5007 - f1_m: 0.3680 - precision_m: 0.2781 - recall_m: 0.5555 - val_loss: 0.6932 - val_acc: 0.5000 - val_f1_m: 0.6621 - val_precision_m: 0.5000 - val_recall_m: 1.0000
Epoch 3/10
3125/3125 [==============================] - 740s 237ms/step - loss: 0.6932 - acc: 0.4963 - f1_m: 0.3678 - precision_m: 0.2773 - recall_m: 0.5584 - val_loss: 0.6932 - val_acc: 0.5000 - val_f1_m: 0.6626 - val_precision_m: 0.5000 - val_recall_m: 1.0000
Epoch 4/10
3125/3125 [==============================] - 345s 110ms/step - loss: 0.6932 - acc: 0.4986 - f1_m: 0.3489 - precision_m: 0.2633 - recall_m: 0.5280 - val_loss: 0.6931 - val_acc: 0.5000 - val_f1_m: 0.0000e

In [14]:
# Visualizing results
training_loss = cnn_history.history['loss']
validation_loss = cnn_history.history['val_loss']
training_accuracy = cnn_history.history['accuracy']
validation_accuracy = cnn_history.history['val_accuracy']
training_precision = cnn_history.history['precision_m']
validation_precision = cnn_history.history['val_precision_m']
epochs = range(1, len(training_loss) + 1)

# Plotting Loss
plt.figure(figsize=(10, 5))
plt.subplot(2, 2, 1)
plt.plot(epochs, training_loss, label='Training loss')
plt.plot(epochs, validation_loss, label='Validation loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting Accuracy
plt.subplot(2, 2, 2)
plt.plot(epochs, training_precision, label='Training Accuracy')
plt.plot(epochs, validation_precision, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting Precision
plt.subplot(2, 2, 3)
plt.plot(epochs, training_precision, label='Training precision')
plt.plot(epochs, validation_precision, label='Validation precision')
plt.title('Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()

plt.tight_layout()
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
  File "C:\Users\sanya\AppData\Local\Temp\ipykernel_6360\1408623295.py", line 4, in <module>
    training_accuracy = cnn_history.history['accuracy']
KeyError: 'accuracy'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
  File "c:\

In [15]:
# Testing set
cnn.evaluate(test_generator)

625/625 [==============================] - 202s 324ms/step - loss: 0.6931 - acc: 0.5000 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00


[0.6931484341621399, 0.5, 0.0, 0.0, 0.0]

Manual testing

In [29]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Real image
img_path = "manual_test_images/Real.jpg"
img = image.load_img(img_path, target_size=(128, 128))
# plt.imshow(img)
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array.reshape(1, 128, 128, 3))
prediction = cnn.predict(img_array)
if prediction[0][0] == 1:
    print("REAL IMAGE")
else:
    print("FAKE IMAGE")


1/1 [==============================] - 1s 754ms/step
FAKE IMAGE


In [30]:
# Fake image(hard)
img_path = "manual_test_images/Hard.jpeg"
img = image.load_img(img_path, target_size=(128, 128))
# plt.imshow(img)
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array.reshape(1, 128, 128, 3))
prediction = cnn.predict(img_array)
if prediction[0][0] == 1:
    print("REAL IMAGE")
else:
    print("FAKE IMAGE")

1/1 [==============================] - 0s 20ms/step
FAKE IMAGE


In [31]:
# Fake image(medium)
img_path = "manual_test_images/Medium.jpg"
img = image.load_img(img_path, target_size=(128, 128))
# plt.imshow(img)
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array.reshape(1, 128, 128, 3))
prediction = cnn.predict(img_array)
if prediction[0][0] == 1:
    print("REAL IMAGE")
else:
    print("FAKE IMAGE")

1/1 [==============================] - 0s 18ms/step
FAKE IMAGE


In [19]:
# Fake image(easy)
img_path = "manual_test_images/Easy.jpg"
img = image.load_img(img_path, target_size=(128, 128))
plt.imshow(img)
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array.reshape(1, 128, 128, 3))
prediction = cnn.predict(img_array)
if prediction[0][0] == 1:
    print("REAL IMAGE")
else:
    print("FAKE IMAGE")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
  File "C:\Users\sanya\AppData\Local\Temp\ipykernel_6360\3922879377.py", line 4, in <module>
    plt.imshow(img)
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\matplotlib\pyplot.py", line 3343, in imshow
    __ret = gca().imshow(
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\matplotlib\pyplot.py", line 2525, in gca
    return gcf().gca()
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\matplotlib\pyplot.py", line 1000, in gcf
    return figure()
  File "c:\Users\sanya\Documents\AI image detection research\generated-image-detection\.env\lib\site-packages\matplotlib\pyplot.py", line 934, in figure
    ma

In [25]:
# Saving model
from datetime import datetime
filename = f'models/cnn_{datetime.now().timestamp()}.h5'
tf.keras.models.save_model(cnn, filename)